# 네이버 뉴스 크롤 만들기
date: 2024-01-17
written by: [Jehwan Kim](github.com/kreimben)

In [1]:
# !pip install -r requirements.txt --upgrade

In [2]:
import traceback
from datetime import datetime, timedelta
from time import sleep

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
end_date = datetime.now()
start_date = end_date - timedelta(days=2)  # 최근 3일간의 데이터를 위한 설정

end_date = end_date.strftime("%Y%m%d")
start_date = start_date.strftime("%Y%m%d")

query = '기사'
url = f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={start_date}&de={end_date}&docid=&nso=so:r,p:from{start_date}to{end_date},a:all&mynews=0&refresh_start=0&related=0"
max_page = 100  # 크롤링을 원하는 최대 페이지 수 지정 # 너무 크면 파일 사이즈가 커짐.

start_date, end_date

('20240116', '20240118')

In [4]:
# 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
titles = []
dates = []
articles = []
article_urls = []
press_companies = []
summaries_map = {}
summaries = []
categories = []
category_kind = {
    '정치': 100, '경제': 101, '사회': 102, '생활/문화': 103, "세계": 104, "IT/과학": 105, '연예': 106, '스포츠': 107
}

# 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
current_call = 1
last_call = (max_page - 1) * 10 + 1  # max_page이 5일 경우 41에 해당 (1페이지는 url에 포함되어 있으므로 1을 빼줌)

# For error rate calulation
errors = []

# For exclude duplicated articles
visit = []

In [5]:
while current_call <= last_call:

    print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))

    url = "https://search.naver.com/search.naver?where=news&query=" + query \
          + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
          + "to" + end_date \
          + "%2Ca%3A&start=" + str(current_call)

    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    urls_list = []
    for urls in source.find_all('a', {'class': "info"}):
        if urls["href"].startswith("https://n.news.naver.com"):
            summaries_map[urls["href"]] = source.find('a', {'class': 'api_txt_lines dsc_txt_wrap'}).get_text()
            urls_list.append(urls["href"])

    for url in urls_list:
        # 중복 기사 제거
        if url in visit:
            continue
        else:
            visit.append(url)

        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
            web_news = requests.get(url, headers=headers).content
            source_news = BeautifulSoup(web_news, 'html.parser')

            if title := source_news.find('h2', {'class': 'media_end_head_headline'}):
                title = title.get_text()
            else:
                title = source_news.find('h2', {'class': 'end_tit'}).get_text()
            print('Processing article : {}'.format(title))

            date = source_news.find('span', {'class': 'media_end_head_info_datestamp_time'}).get_text()

            article = source_news.find('article', {'id': 'dic_area'}).get_text()
            article = article.replace("\n", "")
            article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
            article = article.replace("동영상 뉴스       ", "")
            article = article.replace("동영상 뉴스", "")
            article = article.strip()

            press_company = source_news.find('em', {'class': 'media_end_linked_more_point'}).get_text()

            titles.append(title)
            dates.append(date)
            articles.append(article)
            press_companies.append(press_company)
            article_urls.append(url)
            summaries.append(summaries_map.get(url, ''))

            for k, v in category_kind.items():
                find = f'sid={v}'
                target = url
                if find in target:
                    categories.append(k)
                    break
            else:
                categories.append('N/A')
        except Exception as e:
            print(f'*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {url}')
            print(f'에러 내용 : {e}')
            print(traceback.format_exc())
            errors.append(url)

    # 대량의 데이터를 대상으로 크롤링을 할 때에는 요청 사이에 쉬어주는 타이밍을 넣는 것이 좋습니다.
    sleep(1)
    current_call += 10


1번째 기사글부터 크롤링을 시작합니다.
Processing article : [단독] '보복 운전' 이경 "대리기사 찾았다"…민주 이의신청처리위원회 오늘 회의
Processing article : 문화예술인들, KBS에 故 이선균 관련 기사 삭제 요구
Processing article : "다리 내려달라" 버스기사 요구에 망치로 내리친 20대 중국인
Processing article : 72살 ‘무사고’ 택시기사님, 1명 살리고 하늘로…
Processing article : 한진, 1만 택배기사 대상 '찾아가는 건겅검진서비스'
Processing article : 둔기로 버스기사 폭행한 20대 중국인 검거
Processing article : 번호판 장사 퇴출…화물기사 표준운임 가이드라인 마련
Processing article : '파키스탄, 이란 공습' 기사 읽는 파키스탄 남성
Processing article : ‘30년 무사고’ 70대 택시기사…새 삶 주고 하늘로 떠났다

11번째 기사글부터 크롤링을 시작합니다.
Processing article : AI에게 한동훈·이재명 기사 댓글 요청해보니… [취재후]
Processing article : 전청조 ‘공범’ 주장에…남현희 “사기꾼 말 기사화 그만, 억울하다”
Processing article : 경남도, 법인택시 기사에 처우 개선비 매달 5만원씩 지급
Processing article : 티디아이 플레이, 빅데이터 기반 기사 공급 서비스 '뉴스허브' 론칭
Processing article : 주택가 불 끄고 사라진 택배기사에 ‘감사장’
Processing article : 허식 인천시의장, 이번엔 '한동훈 비난 기사' 공유 논란
Processing article : 온 힘 다해 큰불 막은 택배기사, 소방차 오자 재 뒤집어쓴 채 돌아섰다
Processing article : '정차도 안했는데' 문 개방..서울 버스 운전기사 위험행동 백태

21번째 기사글부터 크롤링을 시작합니다.
Processing ar

In [6]:
# Dataset Length Check
print(f'Titles: {len(titles)}')
print(f'Dates: {len(dates)}')
print(f'Articles: {len(articles)}')
print(f'Article URLs: {len(article_urls)}')
print(f'Press Companies: {len(press_companies)}')
if not (len(titles) == len(dates) == len(articles) == len(article_urls) == len(press_companies)):
    raise ValueError('Dataset Length is not equal')

print(f'{len(errors)} errors occured')

Titles: 405
Dates: 405
Articles: 405
Article URLs: 405
Press Companies: 405
18 errors occured


In [7]:
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모으고 엑셀 파일로 저장합니다.
# 파일명을 result_연도월일_시분.csv 로 지정합니다.
article_df = pd.DataFrame({
    'title': titles,
    'date': dates,
    'document': articles,
    'link': article_urls,
    'press': press_companies,
    'category': categories,
    'summary': summaries
})

article_df.to_csv(f'result_from_{start_date}_end_{end_date}.csv', index=False, encoding='utf-8')

In [8]:
article_df.sort_values(by='date', ascending=True)

,title,date,document,link,press,category,summary
113,"'병립형 회귀' 못 박은 윤재옥 ""민주당, 군소야당 뒤에 숨지 마라""",2024.01.16. 오전 10:01,"""비례연합정당은 꼼수에 불과, 4년 전보다 더 심하게 표심 왜곡할 것""▲ 윤재옥 ...",https://n.news.naver.com/mnews/article/047/000...,오마이뉴스,N/A,겨울철 화재가 증가하고 있는 가운데 CJ대한통운 택배기사가 배송 업무 중 큰 불로 ...
181,"[속보]윤 ""국회, 중대재해처벌법 유예요청에 묵묵부답…중기 여건 감안을""",2024.01.16. 오전 10:06,[서울=뉴시스] 후속기사가 이어집니다▶ 네이버에서 뉴시스 구독하기▶ K-Artpr...,https://n.news.naver.com/mnews/article/003/001...,뉴시스,정치,후속기사가 이어집니다
349,"[속보]윤 ""분양가 상한 주택 실거주 의무 폐지 더는 지체 못해""",2024.01.16. 오전 10:07,[서울=뉴시스] 후속기사가 이어집니다▶ 네이버에서 뉴시스 구독하기▶ K-Artpr...,https://n.news.naver.com/mnews/article/003/001...,뉴시스,정치,18일 오후 2시 현재 인도 SENSEX지수는 전장대비 445.93포인트(0.62%...
356,"[속보]윤 ""산은 부산 이전, 정치적 유불리 넘어 미래 위해 고민을""",2024.01.16. 오전 10:08,[서울=뉴시스] 후속기사가 이어집니다▶ 네이버에서 뉴시스 구독하기▶ K-Artpr...,https://n.news.naver.com/mnews/article/003/001...,뉴시스,정치,이 기사는 2024년 01월 17일 09시 07분 더벨 무료페이지에 표출된 기사입니...
368,"[속보]윤 ""부담금 남발 안돼…91개 부담금 전수조사·원점서 재검토""",2024.01.16. 오전 10:08,[서울=뉴시스] 후속기사가 이어집니다▶ 네이버에서 뉴시스 구독하기▶ K-Artpr...,https://n.news.naver.com/mnews/article/003/001...,뉴시스,정치,후속기사가 이어집니다
...,...,...,...,...,...,...,...
12,"티디아이 플레이, 빅데이터 기반 기사 공급 서비스 '뉴스허브' 론칭",2024.01.18. 오후 3:31,티디아이 플레이 제공티디아이 플레이는 실시간 빅데이터 기반 기사 공급 플랫폼인 '뉴...,https://n.news.naver.com/mnews/article/088/000...,매일신문,IT/과학,댓글 AI 프로그램이 어떻게 포털사이트 기사에 댓글을 다는지 알아보기 위해 취재진이...
117,"[속보]코스피, 3거래일 만에 반등…2440선 마감",2024.01.18. 오후 3:32,[서울=뉴시스] 박은비 기자 = 후속기사가 이어집니다,https://n.news.naver.com/mnews/article/003/001...,뉴시스,경제,후속기사가 이어집니다
92,"문화예술인들, 경찰청 찾아 故 이선균 수사 진상 규명 촉구",2024.01.18. 오후 3:37,KBS에 보도 목적 부합 않는 기사 삭제 요구도연명에 윤여정·송강호·류승룡·황정민 ...,https://n.news.naver.com/mnews/article/277/000...,아시아경제,생활/문화,“안전도 함께 배송하는 동네 지킴이 되겠다” CJ대한통운 택배기사로 일하고 있는 정...
175,"관악구, 장애인 디지털 정보 신문 구독 서비스 도입",2024.01.18. 오후 3:39,[한겨레] 관악구가 장애인들을 위해 ‘디지털 정보 신문 구독 서비스’를 선보인다. ...,https://n.news.naver.com/mnews/article/028/000...,한겨레,사회,이 기사는 2024년01월18일 08시50분에 팜이데일리 프리미엄 콘텐츠로 선공개 ...


In [9]:
article_df.category.isna().sum()
# 카테고리 분류가 제대로 되지 않은 뉴스 기사는 없다.

0

In [10]:
article_df.count().sum()

2835